# PCA

### 요약
뒤에서 소개할 CFA와 같은 의도를 가졌지만, 추구하는 방향성이 다름.           
PCA는 n차원의 변수를 더 낮은 차원의 변수로 설명하고자 하는 방법임,          
따라서 원본 `데이터 포인트`의 분산을 최대한 설명할 수 있는 변수로의 변환이 중요함

In [1]:
import pandas as pd
import pca

In [3]:
D=pd.read_excel("https://data.hossam.kr/E04/diamonds.xlsx")
data=pd.DataFrame(D)

### step1. `데이터 포인트`가 가진 분산을 표시 -> 분산행렬

In [4]:
import numpy as np

In [17]:
clarity_rank='FL>IF>VVS1>VVS2>VS1>VS2>SI1>SI2>I1>I2>I3'
clarity_rank=clarity_rank.split(">")
clarity_rank={y:len(clarity_rank)-x for x,y in enumerate(clarity_rank)}

### 연속형 feature

In [22]:
numeric=['carat','x','y','z','depth','table']
target="price"
category={'cut',"clarity",'color'}

In [25]:
numer_data=data[numeric]
numer_data

,carat,x,y,z,depth,table
0,0.23,3.95,3.98,2.43,61.5,55.0
1,0.21,3.89,3.84,2.31,59.8,61.0
2,0.23,4.05,4.07,2.31,56.9,65.0
3,0.29,4.20,4.23,2.63,62.4,58.0
4,0.31,4.34,4.35,2.75,63.3,58.0
...,...,...,...,...,...,...
53935,0.72,5.75,5.76,3.50,60.8,57.0
53936,0.72,5.69,5.75,3.61,63.1,55.0
53937,0.70,5.66,5.68,3.56,62.8,60.0
53938,0.86,6.15,6.12,3.74,61.0,58.0


In [27]:
numer_data.corr()

,carat,x,y,z,depth,table
carat,1.000000,0.975094,0.951722,0.953387,0.028224,0.181618
x,0.975094,1.000000,0.974701,0.970772,-0.025289,0.195344
y,0.951722,0.974701,1.000000,0.952006,-0.029341,0.183760
z,0.953387,0.970772,0.952006,1.000000,0.094924,0.150929
depth,0.028224,-0.025289,-0.029341,0.094924,1.000000,-0.295779
table,0.181618,0.195344,0.183760,0.150929,-0.295779,1.000000


In [30]:
import statsmodels.api as sm
import sklearn.preprocessing as skpre
import scipy.stats as sps

In [33]:
scaled_numer_data=pd.DataFrame(skpre.scale(numer_data),columns=numer_data.columns)
scaled_numer_data

,carat,x,y,z,depth,table
0,-1.198168,-1.587837,-1.536196,-1.571129,-0.174092,-1.099672
1,-1.240361,-1.641325,-1.658774,-1.741175,-1.360738,1.585529
2,-1.198168,-1.498691,-1.457395,-1.741175,-3.385019,3.375663
3,-1.071587,-1.364971,-1.317305,-1.287720,0.454133,0.242928
4,-1.029394,-1.240167,-1.212238,-1.117674,1.082358,0.242928
...,...,...,...,...,...,...
53935,-0.164427,0.016798,0.022304,-0.054888,-0.662711,-0.204605
53936,-0.164427,-0.036690,0.013548,0.100988,0.942753,-1.099672
53937,-0.206621,-0.063434,-0.047741,0.030135,0.733344,1.137995
53938,0.130927,0.373383,0.337506,0.285204,-0.523105,0.242928


In [39]:
# 이런 방법도 있습니다
scaled_numer_data.T@scaled_numer_data*1/(len(scaled_numer_data.index))
# 원래는 자유도가 (-1)만큼 작아야 야하지만...

,carat,x,y,z,depth,table
carat,1.000000,0.975094,0.951722,0.953387,0.028224,0.181618
x,0.975094,1.000000,0.974701,0.970772,-0.025289,0.195344
y,0.951722,0.974701,1.000000,0.952006,-0.029341,0.183760
z,0.953387,0.970772,0.952006,1.000000,0.094924,0.150929
depth,0.028224,-0.025289,-0.029341,0.094924,1.000000,-0.295779
table,0.181618,0.195344,0.183760,0.150929,-0.295779,1.000000


In [42]:
# eigen_values,eigen_vecotrs
eigen_values,eigen_vectors=np.linalg.eig(scaled_numer_data.T@scaled_numer_data*1/(len(scaled_numer_data.index)))

#### 시각화로 확인

In [54]:
# 이것이 주성분으로 해당 데이터포인트들의 분산을 설명하는 정도
eigen_values/sum(eigen_values)

array([0.65535644, 0.21400867, 0.11389973, 0.0022018 , 0.0079    ,
       0.00663336])

#### 아래는 주성분을 계산하는 다양한 방법의 소개

In [77]:
pd.DataFrame(data[numeric]*eigen_vectors[0]).sum(axis=1)

0       -56.799416
1       -56.447396
2       -54.808065
3       -57.961156
4       -58.646661
           ...    
53935   -56.499289
53936   -57.958206
53937   -58.528958
53938   -56.818173
53939   -57.255482
Length: 53940, dtype: float64

In [78]:
data[numeric]@eigen_vectors[0]

0       -56.799416
1       -56.447396
2       -54.808065
3       -57.961156
4       -58.646661
           ...    
53935   -56.499289
53936   -57.958206
53937   -58.528958
53938   -56.818173
53939   -57.255482
Length: 53940, dtype: float64

여기서 알수있다시피, PCA는 feature의 개수를 보존하려는 성질이 있음